### Langchain: Q & A Over Documents 

Example tool to query product catalog.



In [26]:
# !pip install --upgrade langchain
# !pip install --upgrade bs4

In [27]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [43]:

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
import langchain
langchain.verbose = False


In [47]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [48]:
from langchain.indexes import VectorstoreIndexCreator

In [49]:
# !pip install docarray

In [50]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 4774: character maps to <undefined>

In [ ]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [72]:
response = index.query(query)

AttributeError: module 'langchain' has no attribute 'verbose'

In [73]:
display(Markdown(response))

NameError: name 'response' is not defined

In [74]:
loader = CSVLoader(file_path=file, encoding='utf-8')

In [75]:
docs = loader.load()

In [76]:
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \r\n\r\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \r\n\r\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \r\n\r\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \r\n\r\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [77]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [78]:
embed = embeddings.embed_query("Hi my name is Chris")

In [79]:
print(len(embed))

1536


In [80]:
print(embed[:5])

[-0.017098527401685715, 0.007435472682118416, -0.015752186998724937, -0.037868890911340714, -0.019558658823370934]


In [81]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [82]:
query = "Please suggest a shirt with sunblocking"

In [83]:
docs = db.similarity_search(query)

In [84]:
len(docs)

4

In [85]:
docs[0]

Document(page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \r\n\r\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\r\n\r\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\r\n\r\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\r\n\r\nSun Protection That Won\'t Wear Off\r\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255})

In [86]:
retriever = db.as_retriever()

In [87]:
llm = ChatOpenAI(temperature = 0.0)

In [88]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [89]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.")

AttributeError: module 'langchain' has no attribute 'debug'

In [90]:
display(Markdown(response))

NameError: name 'response' is not defined

In [91]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

AttributeError: module 'langchain' has no attribute 'verbose'

In [92]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [93]:
response = qa_stuff.run(query)

NameError: name 'qa_stuff' is not defined

In [94]:
display(Markdown(response))

NameError: name 'response' is not defined

In [95]:
response = index.query(query, llm=llm)

AttributeError: module 'langchain' has no attribute 'verbose'

In [96]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])